In [6]:
# Script Name: Option_1 (Arbitrage Bot - Clean, Accurate, Manual Execution)
import ccxt
import time
import datetime
from zoneinfo import ZoneInfo
import json
import os

# API Keys (Bybit and Bitget only for this trial)
BYBIT_API_KEY = 'I6m01uNAWiensHbIBA'
BYBIT_API_SECRET = 'vkaviS9xw903NJPncMtHgGfyiWB4mnhSJTyK'
BITGET_API_KEY = 'bg_b0a9d721607623216a87b066a3df34c3'
BITGET_API_SECRET = '6c4a5016562624b6d7f3b1c59563d4a1a573f630e0e49422fbf509399e56918b'
BITGET_API_PASSWORD = 'BgSecure789Trade2025'

# --- CONFIG ---
SPREAD_THRESHOLD = 0.7 / 100
ADD_POSITION_USDT = 15
TRADE_USDT_SIZE = 30
MIN_VOLUME = 50000
CHECK_INTERVAL = 1
LOCAL_TIMEZONE = ZoneInfo("Asia/Jakarta")
TRADE_FILE = 'open_trades.json'

TRADING_FEES = {'Bybit': 0.001, 'Bitget': 0.0008}
futures_markets_cache = {}
exchanges = {}

# --- INIT EXCHANGES ---
bybit = ccxt.bybit({
    'apiKey': BYBIT_API_KEY,
    'secret': BYBIT_API_SECRET,
    'enableRateLimit': True,
    'options': {'defaultType': 'spot'}
})
bitget = ccxt.bitget({
    'apiKey': BITGET_API_KEY,
    'secret': BITGET_API_SECRET,
    'password': BITGET_API_PASSWORD,
    'enableRateLimit': True,
    'options': {'defaultType': 'spot'}
})
exchanges = {'Bybit': bybit, 'Bitget': bitget}


In [ ]:
# --- STORAGE ---
def load_open_trades():
    if os.path.exists(TRADE_FILE):
        with open(TRADE_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_open_trades(trades):
    with open(TRADE_FILE, 'w') as f:
        json.dump(trades, f, indent=2)

open_trades = load_open_trades()

# --- HELPERS ---
def get_price(exchange, symbol):
    try:
        return exchange.fetch_ticker(symbol).get("last")
    except:
        return None

def get_best_bid_ask(exchange, symbol):
    ob = exchange.fetch_order_book(symbol)
    bid = ob['bids'][0][0] if ob['bids'] else None
    ask = ob['asks'][0][0] if ob['asks'] else None
    return bid, ask

def find_futures_symbol(spot_symbol, futures_list):
    base = spot_symbol.split('/')[0]
    return next((f for f in futures_list if f.startswith(f"{base}/USDT") or f.startswith(f"{base}-USDT")), None)

def get_tradable_tokens(exchange, exchange_name):
    try:
        markets = exchange.load_markets()
        tradable = []
        futures_markets_cache[exchange_name] = [s for s in markets if ":USDT" in s or "-USDT-SWAP" in s]
        for s, m in markets.items():
            if exchange_name == "Bitget" and not m.get('spot'): continue
            if '/USDT' in s and ':USDT' not in s and '-USDT-SWAP' not in s:
                try:
                    vol = exchange.fetch_ticker(s).get('quoteVolume', 0)
                    if vol and vol >= MIN_VOLUME:
                        tradable.append(s)
                except: continue
        return tradable
    except: return []

def ask_user_permission(symbol, spread, net_profit):
    ans = input(f"\n🔍 Opportunity: {symbol} | Spread: {spread:.4%} | Net: {net_profit:.4%}. Execute? (yes/no): ").strip().lower()
    return ans == 'yes'

def execute_trade(exchange_name, symbol, usdt_size):
    try:
        exchange = exchanges[exchange_name]
        spot_price = get_price(exchange, symbol)
        if not spot_price:
            print(f"❌ No spot price for {symbol}")
            return

        futures_list = futures_markets_cache.get(exchange_name, [])
        futures_symbol = find_futures_symbol(symbol, futures_list)
        if not futures_symbol:
            print(f"❌ No futures match for {symbol}")
            return

        amount = usdt_size / spot_price
        best_bid, _ = get_best_bid_ask(exchange, symbol)
        order = exchange.create_order(symbol=symbol, type='limit', side='buy', amount=amount, price=best_bid)

        wait_time, filled = 0, 0
        while wait_time < 15:
            o = exchange.fetch_order(order['id'], symbol)
            filled = o['filled']
            if filled > 0: break
            new_price = get_best_bid_ask(exchange, symbol)[0]
            if o['price'] != new_price:
                exchange.cancel_order(order['id'], symbol)
                order = exchange.create_order(symbol=symbol, type='limit', side='buy', amount=amount, price=new_price)
            time.sleep(1)
            wait_time += 1

        if filled == 0:
            exchange.cancel_order(order['id'], symbol)
            print("❌ Spot order not filled. Skipped.")
            return

        if exchange_name == 'Bybit':
            fut = ccxt.bybit({'apiKey': BYBIT_API_KEY, 'secret': BYBIT_API_SECRET, 'options': {'defaultType': 'future'}})
        else:
            fut = ccxt.bitget({'apiKey': BITGET_API_KEY, 'secret': BITGET_API_SECRET, 'password': BITGET_API_PASSWORD, 'options': {'defaultType': 'swap'}})
        fut.load_markets()
        fut.set_leverage(1, futures_symbol)
        fut.create_order(symbol=futures_symbol, type='market', side='sell', amount=filled)

        entry_spread = (get_price(fut, futures_symbol) - spot_price) / spot_price
        trade_id = f"{symbol.replace('/', '')}_{int(time.time())}"
        open_trades[trade_id] = {
            "symbol": symbol,
            "exchange": exchange_name,
            "amount": filled,
            "spot_price": spot_price,
            "futures_price": get_price(fut, futures_symbol),
            "entry_spread": entry_spread,
            "timestamp": datetime.datetime.now().isoformat()
        }
        save_open_trades(open_trades)
        print(f"✅ Executed & saved: {trade_id}")
    except Exception as e:
        print(f"❌ Execution error: {e}")

def check_arbitrage(exchange, exchange_name, symbol):
    spot_price = get_price(exchange, symbol)
    futures_symbol = find_futures_symbol(symbol, futures_markets_cache.get(exchange_name, []))
    if not futures_symbol: return

    futures_price = get_price(exchange, futures_symbol)
    if not spot_price or not futures_price: return

    spread = (futures_price - spot_price) / spot_price
    net_profit = spread - TRADING_FEES[exchange_name] * 2

    timestamp = datetime.datetime.now(LOCAL_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S %Z')
    print(f"🔎 Arbitrage Opportunity on {exchange_name}: {symbol} | {timestamp}")
    print(f"  ✅ Spot Price: {spot_price:.5f} USDT")
    print(f"  ✅ Futures Price: {futures_price:.5f} USDT")
    print(f"  ✅ Spread: {spread:.4%}")
    print(f"  ✅ Net Profit After Fees: {net_profit:.4%}")

    print(f"⚠️  Arbitrage Signal on {exchange_name} - {symbol}")
    print(f"Spread: {spread:.4%} | Net Profit: {net_profit:.4%}")

    for tid, t in open_trades.items():
        if t['symbol'] == symbol and t['exchange'] == exchange_name:
            if spread - t['entry_spread'] >= 0.005:
                print(f"🔁 Spread increased +0.5%, eligible to scale position.")
                if ask_user_permission(symbol, spread, net_profit):
                    execute_trade(exchange_name, symbol, ADD_POSITION_USDT)
            return

    if net_profit >= SPREAD_THRESHOLD:
        if ask_user_permission(symbol, spread, net_profit):
            execute_trade(exchange_name, symbol, TRADE_USDT_SIZE)

def process_tokens(exchange, exchange_name, tokens):
    for token in tokens:
        check_arbitrage(exchange, exchange_name, token)

def main():
    while True:
        try:
            for name, ex in exchanges.items():
                tokens = get_tradable_tokens(ex, name)
                process_tokens(ex, name, tokens)
            time.sleep(CHECK_INTERVAL)
        except Exception as e:
            print(f"❌ Loop error: {e}")
            time.sleep(5)

if __name__ == '__main__':
    main()

🔎 Arbitrage Opportunity on Bybit: BTC/USDT | 2025-04-12 01:30:53 WIB
  ✅ Spot Price: 83975.00000 USDT
  ✅ Futures Price: 83945.60000 USDT
  ✅ Spread: -0.0350%
  ✅ Net Profit After Fees: -0.2350%
⚠️  Arbitrage Signal on Bybit - BTC/USDT
Spread: -0.0350% | Net Profit: -0.2350%
🔎 Arbitrage Opportunity on Bybit: ETH/USDT | 2025-04-12 01:30:53 WIB
  ✅ Spot Price: 1573.61000 USDT
  ✅ Futures Price: 1572.80000 USDT
  ✅ Spread: -0.0515%
  ✅ Net Profit After Fees: -0.2515%
⚠️  Arbitrage Signal on Bybit - ETH/USDT
Spread: -0.0515% | Net Profit: -0.2515%
🔎 Arbitrage Opportunity on Bybit: XRP/USDT | 2025-04-12 01:30:53 WIB
  ✅ Spot Price: 2.06060 USDT
  ✅ Futures Price: 2.05950 USDT
  ✅ Spread: -0.0534%
  ✅ Net Profit After Fees: -0.2534%
⚠️  Arbitrage Signal on Bybit - XRP/USDT
Spread: -0.0534% | Net Profit: -0.2534%
🔎 Arbitrage Opportunity on Bybit: EOS/USDT | 2025-04-12 01:30:53 WIB
  ✅ Spot Price: 0.62980 USDT
  ✅ Futures Price: 0.62940 USDT
  ✅ Spread: -0.0635%
  ✅ Net Profit After Fees: -0.2